# Initial Test

* Aluno: Artur Chiaperini Grover
* Exchange: CEX ([CEX.IO](https://cex.io))

In [20]:
import pickle
import pandas as pd
import numpy as np
import ccxt
import datetime
import requests
import json
import time
import sqlite3
import sqlalchemy
from IPython.display import clear_output

----
## Function definition

In [2]:
def create_date_list(start, end):
    '''
    create_date_list(start, end):
    returns a list of dates from start to end with format 'YYYYMMDD' and type string.
    
    parameters:
    start -> Starting date with format 'YYYY-MM-DD' (string).
    end   -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    start = datetime.datetime.strptime(start, '%Y-%m-%d')
    end = datetime.datetime.strptime(end, '%Y-%m-%d')
    step = datetime.timedelta(days = 1)
    date_list = list()
    while start <= end:
        date_list.append(str(start.date()).replace('-', ''))
        start += step
        
    return date_list

In [3]:
def create_url_list(symbol1, symbol2, start, end):
    '''
    create_url_list(symbol1, symbol2, start, end):
    returns a list of urls from start to end date of the symbol1/symbol2 transaction.
    
    parameters:
    symbol1 -> Cryptocurrency acronym (string).
    symbol2 -> Currency acronym (string).
    start   -> Starting date with format 'YYYY-MM-DD' (string).
    end     -> Ending date with format 'YYYY-MM-DD' (string).
    '''
    
    date_list = create_date_list(start, end)
    url_list = list()
    for date in date_list:
        url = 'http://cex.io/api/ohlcv/hd/{}/{}/{}'.format(date, symbol1, symbol2)
        url_list.append(url)
        
    return url_list

In [23]:
def get_ohlcv(symbol1, symbol2, start, end, data_rate):
    '''
    get_ohlcv(symbol1, symbol2, start, end, data_rate):
    returns a string with all read fetched data from cex.io for the given pair symbol1/symbol2 between
    the starting and ending dates and with data rate (1m, 1h or 1d).
    
    Function has a one second sleep to avoid exploding rate limit of exchange (cex.io).
    Cex rate limit is 600 requests per 10 minutes.
    
    parameters:
    symbol1   -> Cryptocurrency acronym (string).
    symbol2   -> Currency acronym (string).
    start     -> Starting date with format 'YYYY-MM-DD' (string).
    end       -> Ending date with format 'YYYY-MM-DD' (string).
    data_rate -> Data rate, cex.io gives the following options: 'data1m', 'data1h' or 'data1d'.
    '''
    
    url_list = create_url_list(symbol1, symbol2, start, end)
    # time_sleep: guarantees that no more than 60 request are done per second.
    time_sleep = (len(url_list) / 60) + 0.01
    print('Number of urls = {}'.format(len(url_list)))
    print('Time sleep = {}'.format(time_sleep))
    count_input = 0
    data_ohlcv_list = []
    for url in url_list:
        print('Reading {} -----'.format(url))
        ohlcv = eval(requests.get(url).json()[data_rate])
        data_ohlcv_list.append(ohlcv)
        print('Fetch input data size {}'.format(len(ohlcv)))
        count_input += len(ohlcv)
        print('Finished -----')
        time.sleep(time_sleep)
        
    print('Total number of inputs {}'.format(count_input))
    return str([item for sublist in data_ohlcv_list for item in sublist])

----
## Fetching data from Exchange and Saving Pickle File

In [24]:
%%time
cex_eth_usd = get_ohlcv('ETH', 'USD', '2017-01-22', '2018-01-22', 'data1m')

Number of urls = 366
Time sleep = 6.109999999999999
Reading http://cex.io/api/ohlcv/hd/20170122/ETH/USD -----
Fetch input data size 68
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170123/ETH/USD -----
Fetch input data size 33
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170124/ETH/USD -----
Fetch input data size 38
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170125/ETH/USD -----
Fetch input data size 39
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170126/ETH/USD -----
Fetch input data size 36
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170127/ETH/USD -----
Fetch input data size 66
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170128/ETH/USD -----
Fetch input data size 77
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170129/ETH/USD -----
Fetch input data size 30
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170130/ETH/USD -----
Fetch input data size 36
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170131/ETH/USD -----
Fetch in

Fetch input data size 188
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170415/ETH/USD -----
Fetch input data size 114
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170416/ETH/USD -----
Fetch input data size 69
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170417/ETH/USD -----
Fetch input data size 95
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170418/ETH/USD -----
Fetch input data size 157
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170419/ETH/USD -----
Fetch input data size 186
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170420/ETH/USD -----
Fetch input data size 179
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170421/ETH/USD -----
Fetch input data size 153
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170422/ETH/USD -----
Fetch input data size 131
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170423/ETH/USD -----
Fetch input data size 122
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170424/ETH/USD -----
Fetch input 

Fetch input data size 752
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170707/ETH/USD -----
Fetch input data size 905
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170708/ETH/USD -----
Fetch input data size 808
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170709/ETH/USD -----
Fetch input data size 537
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170710/ETH/USD -----
Fetch input data size 922
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170711/ETH/USD -----
Fetch input data size 1238
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170712/ETH/USD -----
Fetch input data size 1095
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170713/ETH/USD -----
Fetch input data size 1053
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170714/ETH/USD -----
Fetch input data size 887
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170715/ETH/USD -----
Fetch input data size 966
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170716/ETH/USD -----
Fetch i

Reading http://cex.io/api/ohlcv/hd/20170927/ETH/USD -----
Fetch input data size 772
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170928/ETH/USD -----
Fetch input data size 766
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170929/ETH/USD -----
Fetch input data size 862
Finished -----
Reading http://cex.io/api/ohlcv/hd/20170930/ETH/USD -----
Fetch input data size 642
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171001/ETH/USD -----
Fetch input data size 630
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171002/ETH/USD -----
Fetch input data size 757
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171003/ETH/USD -----
Fetch input data size 751
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171004/ETH/USD -----
Fetch input data size 537
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171005/ETH/USD -----
Fetch input data size 571
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171006/ETH/USD -----
Fetch input data size 647
Finished -----
Reading ht

Fetch input data size 1376
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171219/ETH/USD -----
Fetch input data size 1431
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171220/ETH/USD -----
Fetch input data size 1431
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171221/ETH/USD -----
Fetch input data size 1404
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171222/ETH/USD -----
Fetch input data size 1423
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171223/ETH/USD -----
Fetch input data size 1367
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171224/ETH/USD -----
Fetch input data size 1291
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171225/ETH/USD -----
Fetch input data size 1294
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171226/ETH/USD -----
Fetch input data size 1229
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171227/ETH/USD -----
Fetch input data size 1220
Finished -----
Reading http://cex.io/api/ohlcv/hd/20171228/ETH/USD -----


In [25]:
file_ohlcv_eth_usd = "ohlcv_eth_usd.pkl"
pickle.dump(cex_eth_usd, open(file_ohlcv_eth_usd, "wb"))

In [ ]:
%%time
cex_eth_eur = get_ohlcv('ETH', 'EUR', '2017-01-22', '2018-01-22', 'data1m')

In [ ]:
file_ohlcv_eth_eur = "ohlcv_eth_eur.pkl"
pickle.dump(cex_eth_eur, open(file_ohlcv_eth_eur, "wb"))

----
## Another topic

In [7]:
df = pd.read_json(cex_ethusd)
file_ohlcv_eth_usd = "ohlcv_eth_usd.pkl"
pickle.dump(cex_eth_usd, open(file_ohlcv_eth_usd, "wb"))df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']

In [9]:
df[0] = datetime.datetime.fromtimestamp(df[0])
df.set_index(0)

KeyError: 0